In [2]:
import numpy as np
from Face_Detection_Recognition.faceRecognition.utils import file_processing,image_processing 
from Face_Detection_Recognition.faceRecognition import face_recognition
import os
import random

In [3]:
resize_width = 160
resize_height = 160

In [4]:
def load_dataset(dataset_path,filename):
    '''
    加载人脸数据库
    :param dataset_path: embedding.npy文件（faceEmbedding.npy）
    :param filename: labels文件路径路径（name.txt）
    :return:
    '''
    compare_emb=np.load(dataset_path)
    names_list=file_processing.read_data(filename,split=None)
    return compare_emb,names_list


In [5]:
def face_recognition_image(model_path,dataset_path, filename,image_path):
    # 加载数据库的数据
    dataset_emb,names_list=load_dataset(dataset_path, filename)
    # 初始化mtcnn人脸检测
    face_detect=face_recognition.Facedetection()
    # 初始化facenet
    face_net=face_recognition.facenetEmbedding(model_path)
 
    predicts=[]
    for img in os.listdir(image_path):
        if os.path.isdir(img) or not img.endswith('.jpg'):
            continue
        path=os.path.join(image_path,img)
        image = image_processing.read_image_gbk(path)
        # 获取 判断标识 bounding_box crop_image
        bboxes, landmarks = face_detect.detect_face(image)
        bboxes, landmarks = face_detect.get_square_bboxes(bboxes, landmarks, fixed="height")
        if bboxes == [] or landmarks == []:
            print("-----no face")
            rand=random.randint(0,49)
            predicts.append([img,str(rand)])
            continue
        print("-----image have {} faces".format(len(bboxes)))
        face_images = image_processing.get_bboxes_image(image, bboxes, resize_height, resize_width)
        face_images = image_processing.get_prewhiten_images(face_images)
        pred_emb=face_net.get_embedding(face_images)
        pred_name,pred_score=compare_embadding(pred_emb, dataset_emb, names_list)
        pred_name=pred_name[0]
        print(img,pred_name)
        predicts.append([img,pred_name])
        # 在图像上绘制人脸边框和识别的结果
    show_info=[ n+':'+str(s)[:5] for n,s in zip(pred_name,pred_score)]
    image_processing.show_image_text("face_recognition", image,bboxes,show_info)
    return predicts


In [6]:

def compare_embadding(pred_emb, dataset_emb, names_list,threshold=0.65):
    # 为bounding_box 匹配标签
    pred_num = len(pred_emb)
    dataset_num = len(dataset_emb)
    pred_name = []
    pred_score=[]
#     print(names_list)
    for i in range(pred_num):
        dist_list = []
        for j in range(dataset_num):
            dist = np.sqrt(np.sum(np.square(np.subtract(pred_emb[i, :], dataset_emb[j, :]))))
            dist_list.append(dist)
        min_value = min(dist_list)
        pred_score.append(min_value)
#         if (min_value > threshold):
#             pred_name.append('unknow')
#         else:
        pred_name.append(names_list[dist_list.index(min_value)])
    return pred_name,pred_score

In [8]:
if __name__=='__main__':
    model_path='20180402-114759/'
    dataset_path='emb/faceEmbedding.npy'
    filename='emb/name.txt'
    image_path='test_mtcnn'
    predicts=face_recognition_image(model_path,dataset_path,filename,image_path)
    
    with open('test_ressss.txt','w') as f:
        for  p in predicts:
            f.write(p[0]+' '+p[1]+'\n')
        

Creating networks and loading parameters


/root/miniconda3/envs/myconda/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Model directory: 20180402-114759/
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from 20180402-114759/model-20180402-114759.ckpt-275
-----image have 1 faces
002475.jpg 30
-----image have 1 faces
002476.jpg 27
-----image have 1 faces
002477.jpg 34
-----image have 1 faces
002478.jpg 14
-----image have 1 faces
002479.jpg 17
-----image have 1 faces
002480.jpg 18
-----image have 1 faces
002481.jpg 44
-----image have 1 faces
002482.jpg 40
-----image have 1 faces
002483.jpg 11
-----image have 1 faces
002484.jpg 3
-----image have 1 faces
002485.jpg 29
-----image have 1 faces
002486.jpg 33
-----image have 1 faces
002487.jpg 37
-----image have 1 faces
002488.jpg 15
-----image have 1 faces
002489.jpg 12
-----image have 1 faces
002490.jpg 40
-----image have 1 faces
002491.jpg 42
-----image have 1 faces
002492.jpg 48
-----image have 1 faces
002493.jpg 36
-----image have 1 faces
002494.jpg 17
-----image have 1 faces
002

In [ ]:
print(predicts)

In [7]:
!python eval.py

ACC: 0.9523


In [9]:
!cp test_res3.txt /mnt/hw4/